<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/3%20-%20Funzioni%20di%20costo%2C%20attivazione%20e%20ottimizzazione/learningrate_momentum_rmsprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Rate e Momentum
In questo notebook vedremo come utilizzare Learning Rate e Momentum per migliorare la fase di addestramento di una Rete Neurale

## Importiamo i Moduli

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from time import time

## Prepariamo i Dati

In [ ]:
def load_data(dataset, num_samples=None):
  images = []
  labels = []

  for i, example in enumerate(tfds.as_numpy(dataset)):

    if(num_samples!=None and i>=num_samples):
      break

    images.append(example["image"])
    labels.append(example["label"])

  images = np.array(images)
  labels = np.array(labels)

  return images, labels


dataset = tfds.load('fashion_mnist', split='train', shuffle_files=True)
images, labels = load_data(dataset)
num_classes = np.unique(labels).shape[0]

X = images.reshape(images.shape[0], images.shape[1]*images.shape[2])
y = tf.one_hot(labels, num_classes).numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train = X_train/255
X_test = X_test/255

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteZRAIMQ/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.0.incompleteZRAIMQ/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.0. Subsequent calls will reuse this data.


## Creiamo la Rete Neurale
Definiamo due funzioni, una per definire l'architettura delle Rete e un'altra per creare un grafico della Log Loss.

In [ ]:
def build_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=[X_train.shape[1]]))
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

  return model


def plot_loss_chart(title, figsize=(14,10), validation=True):

  plt.figure(figsize=figsize)
  plt.title(title)
  plt.xlabel("Epoca")
  plt.ylabel("Log-Loss")
  
  plt.plot(model.history.history['loss'], label="Training Loss")

  if(validation):
    plt.plot(model.history.history['val_loss'], label="Validation Loss")

  plt.legend()

## Learning Rate
Il Learning Rate ci permette di controllare la dimensione di ogni step del Gradient Descent. Per  impostare il learning rate possiamo definire l'algoritmo di ottimizzazione tramite una delle apposite classi di tf.keras e utilizzare il parametro *learning_rate*.

#### Learning Rate troppo piccolo
Un valore del Learning Rate troppo piccolo può rendere eccessivamente piccoli gli step del Gradient Descent, rallentando la fase di addestramento in maniera eccessiva.

In [ ]:
model = build_model()

sgd = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 155ms/step - loss: 1.8880 - accuracy: 0.5695 - val_loss: 1.8911 - val_accuracy: 0.5654
Epoch 2/10
1/1 [==============================] - 0s 78ms/step - loss: 1.8867 - accuracy: 0.5700 - val_loss: 1.8898 - val_accuracy: 0.5657
Epoch 3/10
1/1 [==============================] - 0s 79ms/step - loss: 1.8854 - accuracy: 0.5705 - val_loss: 1.8885 - val_accuracy: 0.5663
Epoch 4/10
1/1 [==============================] - 0s 84ms/step - loss: 1.8841 - accuracy: 0.5707 - val_loss: 1.8873 - val_accuracy: 0.5670
Epoch 5/10
1/1 [==============================] - 0s 80ms/step - loss: 1.8828 - accuracy: 0.5712 - val_loss: 1.8860 - val_accuracy: 0.5678
Epoch 6/10
1/1 [==============================] - 0s 82ms/step - loss: 1.8815 - accuracy: 0.5717 - val_loss: 1.8847 - val_accuracy: 0.5682
Epoch 7/10
1/1 [==============================] - 0s 85ms/step - loss: 1.8803 - accuracy: 0.5724 - val_loss: 1.8835 - val_accuracy: 0.5685
Epoch 8/10
1/1 [==========

### Learning Rate troppo grande
Un valore troppo grande del Learning Rate rende gli step del Gradient Descent troppo larghi, facendo divergere il modello.

In [ ]:
model = build_model()

sgd = tf.keras.optimizers.SGD(learning_rate=1)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 150ms/step - loss: 2.4167 - accuracy: 0.0883 - val_loss: 2.3971 - val_accuracy: 0.2297
Epoch 2/10
1/1 [==============================] - 0s 82ms/step - loss: 2.3803 - accuracy: 0.2350 - val_loss: 4.5815 - val_accuracy: 0.0999
Epoch 3/10
1/1 [==============================] - 0s 82ms/step - loss: 4.6311 - accuracy: 0.1016 - val_loss: 8.0767 - val_accuracy: 0.1975
Epoch 4/10
1/1 [==============================] - 0s 78ms/step - loss: 8.1899 - accuracy: 0.1981 - val_loss: 7.7897 - val_accuracy: 0.1004
Epoch 5/10
1/1 [==============================] - 0s 83ms/step - loss: 7.8442 - accuracy: 0.0964 - val_loss: 152.7688 - val_accuracy: 0.0972
Epoch 6/10
1/1 [==============================] - 0s 78ms/step - loss: 152.7806 - accuracy: 0.1012 - val_loss: 89.1033 - val_accuracy: 0.0990
Epoch 7/10
1/1 [==============================] - 0s 75ms/step - loss: 90.7148 - accuracy: 0.1004 - val_loss: 304.4587 - val_accuracy: 0.0567
Epoch 8/10
1/1 [==

### Learning Rate ottimale
Il valore ottimale del Learning Rate varia da problema a problema e andrebbe cercato in una scala logaritmica che va da 0.0001 a 10 (0.0001, 0.001, 0.01, 0.1, 1, 10).

In [ ]:
model = build_model()

sgd = tf.keras.optimizers.SGD(learning_rate=0.1)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 154ms/step - loss: 2.3295 - accuracy: 0.0727 - val_loss: 2.2001 - val_accuracy: 0.1599
Epoch 2/10
1/1 [==============================] - 0s 82ms/step - loss: 2.1973 - accuracy: 0.1630 - val_loss: 2.1078 - val_accuracy: 0.2368
Epoch 3/10
1/1 [==============================] - 0s 77ms/step - loss: 2.1041 - accuracy: 0.2397 - val_loss: 2.0203 - val_accuracy: 0.3551
Epoch 4/10
1/1 [==============================] - 0s 79ms/step - loss: 2.0160 - accuracy: 0.3616 - val_loss: 1.9339 - val_accuracy: 0.4155
Epoch 5/10
1/1 [==============================] - 0s 75ms/step - loss: 1.9290 - accuracy: 0.4249 - val_loss: 1.8461 - val_accuracy: 0.4906
Epoch 6/10
1/1 [==============================] - 0s 82ms/step - loss: 1.8404 - accuracy: 0.4970 - val_loss: 1.7576 - val_accuracy: 0.5382
Epoch 7/10
1/1 [==============================] - 0s 79ms/step - loss: 1.7513 - accuracy: 0.5419 - val_loss: 1.6686 - val_accuracy: 0.5811
Epoch 8/10
1/1 [==========

## Learning Rate Dinamico
Una scelta intelligente è quella di far variare il valore del Learning Rate durante l'addestramento, rendendolo più piccolo man mano che il modello inizia a convergere. Questa tecnica è conosciuta come **Learning Rate Decay**. Esistono diverse tecniche per implementare il Learning Rate Decay, vediamone due.

### Exponential Decay
Questa tecnica riduce il Learning Rate in maniera esponenziale utilizzando il seguente algoritmo:
<br><br>
*initial_learning_rate * decay_rate ^ (step / decay_steps)*
<br><br>


In [ ]:
model = build_model()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.9)

sgd = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 166ms/step - loss: 2.3818 - accuracy: 0.0941 - val_loss: 2.1901 - val_accuracy: 0.1596
Epoch 2/10
1/1 [==============================] - 0s 86ms/step - loss: 2.1871 - accuracy: 0.1637 - val_loss: 2.0611 - val_accuracy: 0.3228
Epoch 3/10
1/1 [==============================] - 0s 83ms/step - loss: 2.0584 - accuracy: 0.3289 - val_loss: 1.9402 - val_accuracy: 0.4764
Epoch 4/10
1/1 [==============================] - 0s 83ms/step - loss: 1.9372 - accuracy: 0.4800 - val_loss: 1.8226 - val_accuracy: 0.5561
Epoch 5/10
1/1 [==============================] - 0s 91ms/step - loss: 1.8193 - accuracy: 0.5620 - val_loss: 1.7084 - val_accuracy: 0.5973
Epoch 6/10
1/1 [==============================] - 0s 90ms/step - loss: 1.7047 - accuracy: 0.6033 - val_loss: 1.5996 - val_accuracy: 0.6148
Epoch 7/10
1/1 [==============================] - 0s 89ms/step - loss: 1.5954 - accuracy: 0.6220 - val_loss: 1.4976 - val_accuracy: 0.6318
Epoch 8/10
1/1 [==========

### Inverse Time Decay
Un'alternativa all'Exponential Decay è l'Inverse Time Decay, che sistema il valore del Learning Rate utilizzando il seguente algoritmo:
<br><br>
*initial_learning_rate / (1 + decay_rate * step / decay_step)*
<br><br>

In [ ]:
model = build_model()

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.9)

sgd = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 154ms/step - loss: 2.3592 - accuracy: 0.0856 - val_loss: 2.1719 - val_accuracy: 0.2756
Epoch 2/10
1/1 [==============================] - 0s 77ms/step - loss: 2.1706 - accuracy: 0.2837 - val_loss: 2.0513 - val_accuracy: 0.3955
Epoch 3/10
1/1 [==============================] - 0s 74ms/step - loss: 2.0489 - accuracy: 0.4028 - val_loss: 1.9410 - val_accuracy: 0.4451
Epoch 4/10
1/1 [==============================] - 0s 75ms/step - loss: 1.9376 - accuracy: 0.4506 - val_loss: 1.8329 - val_accuracy: 0.4978
Epoch 5/10
1/1 [==============================] - 0s 76ms/step - loss: 1.8286 - accuracy: 0.5040 - val_loss: 1.7256 - val_accuracy: 0.5612
Epoch 6/10
1/1 [==============================] - 0s 91ms/step - loss: 1.7207 - accuracy: 0.5689 - val_loss: 1.6187 - val_accuracy: 0.6066
Epoch 7/10
1/1 [==============================] - 0s 89ms/step - loss: 1.6136 - accuracy: 0.6129 - val_loss: 1.5161 - val_accuracy: 0.6297
Epoch 8/10
1/1 [==========

## RMSProp e ADAM

In [ ]:
model = build_model()

model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 152ms/step - loss: 2.3621 - accuracy: 0.1197 - val_loss: 2.0308 - val_accuracy: 0.3428
Epoch 2/10
1/1 [==============================] - 0s 307ms/step - loss: 2.0357 - accuracy: 0.3410 - val_loss: 1.8353 - val_accuracy: 0.3146
Epoch 3/10
1/1 [==============================] - 0s 77ms/step - loss: 1.8238 - accuracy: 0.3189 - val_loss: 2.0341 - val_accuracy: 0.3249
Epoch 4/10
1/1 [==============================] - 0s 77ms/step - loss: 2.0338 - accuracy: 0.3210 - val_loss: 1.8158 - val_accuracy: 0.3793
Epoch 5/10
1/1 [==============================] - 0s 80ms/step - loss: 1.8199 - accuracy: 0.3781 - val_loss: 1.4766 - val_accuracy: 0.5065
Epoch 6/10
1/1 [==============================] - 0s 75ms/step - loss: 1.4629 - accuracy: 0.5099 - val_loss: 1.3408 - val_accuracy: 0.5217
Epoch 7/10
1/1 [==============================] - 0s 73ms/step - loss: 1.3358 - accuracy: 0.5260 - val_loss: 1.1121 - val_accuracy: 0.6392
Epoch 8/10
1/1 [=========

In [ ]:
model = build_model()

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 163ms/step - loss: 2.2950 - accuracy: 0.1314 - val_loss: 1.9431 - val_accuracy: 0.4178
Epoch 2/10
1/1 [==============================] - 0s 73ms/step - loss: 1.9368 - accuracy: 0.4243 - val_loss: 1.6477 - val_accuracy: 0.5544
Epoch 3/10
1/1 [==============================] - 0s 93ms/step - loss: 1.6392 - accuracy: 0.5646 - val_loss: 1.3896 - val_accuracy: 0.6207
Epoch 4/10
1/1 [==============================] - 0s 89ms/step - loss: 1.3807 - accuracy: 0.6278 - val_loss: 1.1677 - val_accuracy: 0.6692
Epoch 5/10
1/1 [==============================] - 0s 76ms/step - loss: 1.1599 - accuracy: 0.6729 - val_loss: 1.0049 - val_accuracy: 0.6849
Epoch 6/10
1/1 [==============================] - 0s 75ms/step - loss: 1.0005 - accuracy: 0.6885 - val_loss: 0.9027 - val_accuracy: 0.6945
Epoch 7/10
1/1 [==============================] - 0s 76ms/step - loss: 0.9002 - accuracy: 0.6998 - val_loss: 0.8317 - val_accuracy: 0.6962
Epoch 8/10
1/1 [==========

In [ ]:
model = build_model()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.9)

adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=X_train.shape[0],
          validation_data=(X_test, y_test), validation_batch_size = X_test.shape[0], 
          epochs=10, callbacks=[tf.keras.callbacks.History()])

Epoch 1/10
1/1 [==============================] - 0s 169ms/step - loss: 2.3244 - accuracy: 0.0919 - val_loss: 2844.4353 - val_accuracy: 0.1034
Epoch 2/10
1/1 [==============================] - 0s 78ms/step - loss: 2848.1594 - accuracy: 0.0985 - val_loss: 1472.3868 - val_accuracy: 0.0987
Epoch 3/10
1/1 [==============================] - 0s 75ms/step - loss: 1461.6931 - accuracy: 0.1006 - val_loss: 302.6365 - val_accuracy: 0.0988
Epoch 4/10
1/1 [==============================] - 0s 78ms/step - loss: 303.1682 - accuracy: 0.1006 - val_loss: 53.7579 - val_accuracy: 0.1129
Epoch 5/10
1/1 [==============================] - 0s 75ms/step - loss: 53.7307 - accuracy: 0.1149 - val_loss: 9.5130 - val_accuracy: 0.2023
Epoch 6/10
1/1 [==============================] - 0s 71ms/step - loss: 9.3106 - accuracy: 0.1980 - val_loss: 5.8453 - val_accuracy: 0.1360
Epoch 7/10
1/1 [==============================] - 0s 76ms/step - loss: 5.7535 - accuracy: 0.1341 - val_loss: 2.7453 - val_accuracy: 0.2049
Epoch 8/